In [1]:
import numpy as np
import sys


class NeuralNetMLP(object):
    def __init__(self, n_hidden=30,
                 l2=0., epochs=100, eta=0.001,
                 shuffle=True, minibatch_size=1, seed=None):

        self.random = np.random.RandomState(seed)
        self.n_hidden = n_hidden
        self.l2 = l2
        self.epochs = epochs
        self.eta = eta
        self.shuffle = shuffle
        self.minibatch_size = minibatch_size

    def _onehot(self, y, n_classes):
    
        onehot = np.zeros((n_classes, y.shape[0]))
        for idx, val in enumerate(y.astype(int)):
            onehot[val, idx] = 1.
        return onehot.T

    def _sigmoid(self, z):
        return 1. / (1. + np.exp(-np.clip(z, -250, 250)))

    def _forward(self, X):
        z_h = np.dot(X, self.w_h) + self.b_h
        a_h = self._sigmoid(z_h)
        z_out = np.dot(a_h, self.w_out) + self.b_out
        a_out = self._sigmoid(z_out)

        return z_h, a_h, z_out, a_out

    def _compute_cost(self, y_enc, output):
        L2_term = (self.l2 *
                   (np.sum(self.w_h ** 2.) +
                    np.sum(self.w_out ** 2.)))

        term1 = -y_enc * (np.log(output))
        term2 = (1. - y_enc) * np.log(1. - output)
        cost = np.sum(term1 - term2) + L2_term
        return cost

    def predict(self, X):
        z_h, a_h, z_out, a_out = self._forward(X)
        y_pred = np.argmax(z_out, axis=1)
        return y_pred

    def fit(self, X_train, y_train, X_valid, y_valid):

        n_output = np.unique(y_train).shape[0] 
        n_features = X_train.shape[1]
        self.b_h = np.zeros(self.n_hidden)
        self.w_h = self.random.normal(loc=0.0, scale=0.1,
                                      size=(n_features, self.n_hidden))
        self.b_out = np.zeros(n_output)
        self.w_out = self.random.normal(loc=0.0, scale=0.1,
                                        size=(self.n_hidden, n_output))

        epoch_strlen = len(str(self.epochs))
        self.eval_ = {'cost': [], 'train_acc': [], 'valid_acc': []}
        y_train_enc = self._onehot(y_train, n_output)
        
        for i in range(self.epochs):
            indices = np.arange(X_train.shape[0])

            if self.shuffle:
                self.random.shuffle(indices)

            for start_idx in range(0, indices.shape[0] - self.minibatch_size +
                                   1, self.minibatch_size):
                batch_idx = indices[start_idx:start_idx + self.minibatch_size]
                z_h, a_h, z_out, a_out = self._forward(X_train[batch_idx])
                sigma_out = a_out - y_train_enc[batch_idx]
                sigmoid_derivative_h = a_h * (1. - a_h)
                sigma_h = (np.dot(sigma_out, self.w_out.T) *
                           sigmoid_derivative_h)
                grad_w_h = np.dot(X_train[batch_idx].T, sigma_h)
                grad_b_h = np.sum(sigma_h, axis=0)
                grad_w_out = np.dot(a_h.T, sigma_out)
                grad_b_out = np.sum(sigma_out, axis=0)
                delta_w_h = (grad_w_h + self.l2*self.w_h)
                delta_b_h = grad_b_h 
                self.w_h -= self.eta * delta_w_h
                self.b_h -= self.eta * delta_b_h
                delta_w_out = (grad_w_out + self.l2*self.w_out)
                delta_b_out = grad_b_out  
                self.w_out -= self.eta * delta_w_out
                self.b_out -= self.eta * delta_b_out
            z_h, a_h, z_out, a_out = self._forward(X_train)
            
            cost = self._compute_cost(y_enc=y_train_enc,
                                      output=a_out)

            y_train_pred = self.predict(X_train)
            y_valid_pred = self.predict(X_valid)

            train_acc = ( (np.sum(y_train_pred==y_train.T )).astype(np.float) / X_train.shape[0])
            
            
            valid_acc = ((np.sum(y_valid.T == y_valid_pred)).astype(np.float) / X_valid.shape[0])

            sys.stderr.write('\r%0*d/%d  '
                             '| 훈련/검증 정확도: %.2f%%/%.2f%% ' %
                             (epoch_strlen, i+1, self.epochs, 
                              train_acc*100, valid_acc*100))
            sys.stderr.flush()

            self.eval_['cost'].append(cost)
            self.eval_['train_acc'].append(train_acc)
            self.eval_['valid_acc'].append(valid_acc)

        return self

In [2]:
import pandas as pd
import math 
from sklearn.model_selection import train_test_split
df = pd.read_csv('squat_o.csv')
X = df.iloc[:,1:]
X = X.values.tolist()  ## DataFrame을 List값으로 추출
array = []
for i in range(0,500): ## 좌표값을 math.atan2(y,x) * 180 / math.pi로 두 점 사이의 각도값 추출
    array.append([
                math.atan2(X[i][5]-X[i][3],X[i][4]-X[i][2]) *180 / math.pi,
                math.atan2(X[i][7]-X[i][5],X[i][4]-X[i][6]) *180 / math.pi,
                math.atan2(X[i][9]-X[i][7],X[i][8]-X[i][6]) *180 / math.pi
                 ,1]
                )    
result_o = pd.DataFrame(array) ## List를 DataFrame으로 변환
df = pd.read_csv('squat_x.csv')
X = df.iloc[:,1:]
X = X.values.tolist()
array = []
for i in range(0,500): ## 각도값 추출 
    array.append([
                math.atan2(X[i][5]-X[i][3],X[i][4]-X[i][2]) *180 / math.pi,
                math.atan2(X[i][7]-X[i][5],X[i][4]-X[i][6]) *180 / math.pi,
                math.atan2(X[i][9]-X[i][7],X[i][8]-X[i][6]) *180 / math.pi
                 ,0]
                )
    
result_x = pd.DataFrame(array) ## List를 DataFrame으로 변환
result_ox = result_o.append(result_x) ## DataFrame 두개를 합쳐서 새로운 DataFrame 정의
X = result_ox.iloc[0:,:3].values ## 입력 Data 추출
y = result_ox.iloc[0:,[3]].values  ## 정답 Lable 추출

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,
                                                    test_size = 0.2,## Dataset을 무작위로 섞어 분리
                                                    random_state=1)
n_epochs = 200

nn = NeuralNetMLP(n_hidden=50, 
                  l2=0, 
                  epochs=n_epochs, 
                  eta=0.0005,
                  minibatch_size=10, 
                  shuffle=True
                  )

nn.fit(X_train,y_train,X_test,y_test)

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,
                                                    test_size = 0.2,## Dataset을 무작위로 섞어 분리
                                                    random_state=1)
n_epochs = 200

nn = NeuralNetMLP(n_hidden=50, 
                  l2=0, 
                  epochs=n_epochs, 
                  eta=0.0005,
                  minibatch_size=10, 
                  shuffle=True
                  )

nn.fit(X_train,y_train,X_test,y_test)



200/200  | 훈련/검증 정확도: 90.25%/92.50% 

In [5]:
y_test
cnt_T =0
cnt_F =0
X_test_pred = nn.predict(X_test)
for i in range(0,200):
    if y_test.T[0][i] == 0:
        if y_test.T[0][i] == X_test_pred[i]:
            cnt_F +=1
    if y_test.T[0][i] == 1:
        if y_test.T[0][i] == X_test_pred[i]:
            cnt_T +=1
            
            
print("총 양성, 음성 각 100개에 대하여","양성 정답 :" ,cnt_T,"음성 정답 :", cnt_F)


총 양성, 음성 각 100개에 대하여 양성 정답 : 90 음성 정답 : 95
